In [1]:
import modelinference
import prompts
import importlib
import datetime
import torch
from huggingface_hub import login


from accelerate import Accelerator, notebook_launcher

In [2]:
importlib.reload(modelinference)

<module 'modelinference' from '/project/modelinference.py'>

In [3]:
# Log into Huggingface

with open('pass.txt') as p:
    hf_login = p.read()
    
hf_login = hf_login[hf_login.find('=')+1:hf_login.find('\n')]
login(hf_login, add_to_git_credential=False)

In [4]:
torch.cuda.device_count()

2

In [5]:

run_config = {
    'model_hf_id': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-14B',
    'model_s3_loc': 'deepseek14Q',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['CoT'],
    'multi-gpu':True,
    'dataset': 'data_annual_pit_spx',
    'data_location': 'data_annual_pit_spx.json'
}

run_config = {
    'model_hf_id': 'Qwen/Qwen2.5-7B-Instruct',
    'model_s3_loc': 'qwen',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['BASE'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu',
    'data_location': 'data_quarterly_pit_indu.json'
}

run_config = {
    'model_hf_id': 'meta-llama/Llama-3.2-3B-Instruct',
    'model_s3_loc': 'llama',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['BASE'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu',
    'data_location': 'data_quarterly_pit_indu.json'
}




run_name = f"{run_config['model_s3_loc']} - {run_config['dataset']}"

In [6]:
# ir = modelinference.InferenceRun(run_name, run_config)
# ir.run_multi_gpu()

In [7]:
def inference_multi_gpu():
    ir = modelinference.InferenceRun(run_name, run_config)
    ir.run_multi_gpu()

In [8]:
notebook_launcher(inference_multi_gpu, num_processes=torch.cuda.device_count())

Launching training on 2 GPUs.
llama
llama


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading Data
Memory footprint: 6.4 GB
waiting...
waiting...
starting backtest...starting backtest...



  0%|          | 0/4 [00:00<?, ?it/s]

...

running model...


100%|██████████| 4/4 [00:58<00:00, 14.69s/it]


Finished run


[2025-02-22 18:01:18,093] torch.distributed.elastic.agent.server.api: [WARNING] Received 2 death signal, shutting down workers
[2025-02-22 18:01:18,095] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 3076 via signal SIGINT
[2025-02-22 18:01:18,096] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 3077 via signal SIGINT
[2025-02-22 18:01:48,114] torch.distributed.elastic.multiprocessing.api: [WARNING] Unable to shutdown process 3076 via 2, forcefully exiting via 9


SignalException: Process 3001 got signal: 2

In [10]:
ir = modelinference.InferenceRun(run_name, run_config)

In [13]:
p1 = ir.create_all_prompts()

In [12]:
len(prompts)

896

In [14]:
model = ir.load_model_from_storage(ir.model_s3_loc)

llama


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
from transformers import AutoTokenizer

In [17]:
tokenizer = AutoTokenizer.from_pretrained(ir.model_hf_id)

In [18]:
ir.run_model(p1[0]['prompt'],tokenizer,model)

'To make a buy, sell or hold decision, I will compute some common financial ratios.\n\n```json\n{\n  "decision": "BUY",\n  "confidence score": 80,\n  "reason": "Price-to-Earnings ratio is higher than the industry average and the company has shown a steady increase in revenue and net income over the past few years."\n}\n```\n\nI computed the following financial ratios:\n\n1. Price-to-Earnings (P/E) ratio:\n   The P/E ratio is calculated by dividing the current price by the EPS. The current price is 136.188257 and the EPS is 1.640000e+00.\n   P/E ratio = 136.188257 / 1.640000e+00 = 83.04\n\n2. Revenue Growth:\n   The revenue has been steadily increasing over the past few years.\n\n3. Net Income Growth:\n   The net income has been steadily increasing over the past few years.\n\n4. Return on Equity (ROE):\n   ROE is calculated by dividing net income by total equity.\n   ROE = 1.012600e+10 / 1.012600e+10 = 1.00\n\n5. Debt-to-Equity ratio:\n   The debt-to-equity ratio is calculated by dividi

In [9]:
import utils.modelHelper as mh
import importlib
importlib.reload(mh)

<module 'utils.modelHelper' from '/project/utils/modelHelper.py'>

In [10]:
helper = mh.ModelHelper('tmp/fs')

In [11]:
helper.clear_folder('llama')
#helper.clear_folder(f'Data/{run_name}')

In [6]:
import os

In [7]:
print(os.)

NameError: name 'ROOT_DIR' is not defined

In [10]:
os.path.abspath(os.getcwd())

'/project'

In [ ]:
with open()